# Chelsa: Environmental Factors Download and Cropping

This section downloads environmental factor data from the CHELSA dataset. The code retrieves data files for various variables (`clt`, `hurs`, `pr`, `rsds`, `sfcWind`, `tas`) for each month (01 to 12) from 2000 to 2019 using the `requests` module. The downloaded files are saved into subfolders under `/work/klok0126/Chelsa/`. After downloading, the files are cropped using GDAL, and the cropped files are saved in folders with an underscore appended (e.g. `/work/klok0126/Chelsa/clt_/`).

**File Location Example:**
- For `clt`: `/work/klok0126/Chelsa/clt_/CHELSA_clt_01_2009_V.2.1.tif`
- For `hurs`: `/work/klok0126/Chelsa/hurs_/CHELSA_hurs_06_2015_V.2.1.tif`
- For `pr`: `/work/klok0126/Chelsa/pr_/CHELSA_pr_12_2012_V.2.1.tif`
- For `rsds`: `/work/klok0126/Chelsa/rsds_/CHELSA_rsds_03_2018_V.2.1.tif`
- For `sfcWind`: `/work/klok0126/Chelsa/sfcWind_/CHELSA_sfcWind_09_2011_V.2.1.tif`
- For `tas`: `/work/klok0126/Chelsa/tas_/CHELSA_tas_07_2013_V.2.1.tif`

In [ ]:
import requests
import time
import os

# List of environmental variables to download
env_list = ['clt', 'hurs', 'pr', 'rsds', 'sfcWind', 'tas']
months = range(1, 13)
years = range(2000, 2020)

# Create a combined list of [variable, month, year]
combined_list = []
for item1 in env_list:
    for item2 in months:
        for item3 in years:
            combined_list.append([item1, item2, item3])
print(f'maximum index: {len(combined_list) - 1}')

start_idx = input('start index for downloading: ')

if start_idx == '':
    start_idx = 0

# Base directory where files will be saved
dir_base = '/work/klok0126/Chelsa'

# Link pattern for generating download URLs
link_pattern = dict(
    dir_base = 'https://os.zhdk.cloud.switch.ch/envicloud/chelsa/chelsa_V2/GLOBAL/monthly', 
    clt = 'clt/CHELSA_clt_[MONTH]_[YEAR]_V.2.1.tif', 
    cmi = 'cmi/CHELSA_cmi_[MONTH]_[YEAR]_V.2.1.tif', 
    hurs = 'hurs/CHELSA_hurs_[MONTH]_[YEAR]_V.2.1.tif', 
    pet = 'pet/CHELSA_pet_penman_[MONTH]_[YEAR]_V.2.1.tif', 
    pr = 'pr/CHELSA_pr_[MONTH]_[YEAR]_V.2.1.tif', 
    rsds = 'rsds/CHELSA_rsds_[YEAR]_[MONTH]_V.2.1.tif', 
    sfcWind = 'sfcWind/CHELSA_sfcWind_[MONTH]_[YEAR]_V.2.1.tif', 
    tas = 'tas/CHELSA_tas_[MONTH]_[YEAR]_V.2.1.tif', 
    tasmax = 'tasmax/CHELSA_tasmax_[MONTH]_[YEAR]_V.2.1.tif', 
    tasmin = 'tasmin/CHELSA_tasmin_[MONTH]_[YEAR]_V.2.1.tif', 
    vpd = 'vpd/CHELSA_vpd_[MONTH]_[YEAR]_V.2.1.tif'
)

# Download the CHELSA dataset files based on the combined list
for [env, month, year] in sorted(combined_list)[int(start_idx):]:
    
    # Create directory for the environmental variable if it doesn't exist
    if not os.path.exists(f'{dir_base}/{env}'):
        os.makedirs(f'{dir_base}/{env}')
    
    # Replace placeholders in the URL with actual month and year values
    link_itr = f'{link_pattern[env]}'.replace('[MONTH]', f'{month:02d}').replace('[YEAR]', str(year))
    link_full = f'{link_pattern["dir_base"]}/{link_itr}'
    
    # Check if the file already exists
    if os.path.exists(f'{dir_base}/{link_itr}'):
        print(f"{link_itr} has already existed.")
        break
    
    print(f'Accessing file: {link_itr}')
    response = requests.get(link_full)
    if response.status_code == 200:
        with open(f'{dir_base}/{link_itr}', 'wb') as file:
            print(f"Downloaded.")
            file.write(response.content)
        time.sleep(3)
    else:
        print(f"Cannot download link: {link_itr}.")
    response = None
print("All links are downloaded.")

In [ ]:
from osgeo import gdal, gdalconst
for env in env_list:
    dir_base = f'/work/klok0126/Chelsa/{env}/'
    dir_dst = f'/work/klok0126/Chelsa/{env}_/'
    if not os.path.exists(dir_dst):
        os.makedirs(dir_dst)

    # Crop each file in the environmental variable folder using GDAL
    for fname in os.listdir(dir_base):
        print(fname)
        ds = gdal.Open(os.path.join(dir_base, fname), gdalconst.GA_ReadOnly)
        gdal.Translate(os.path.join(dir_dst, fname), ds, projWin=[119.5, 25.5, 122.5, 21.5])
        ds = None

# Land Cover Dataset

To download the Land Cover dataset, follow these steps:

- Visit the [ESA Land Cover](https://cds.climate.copernicus.eu/datasets/satellite-land-cover?tab=download) webpage.
- Sign in using your account credentials.
- Select the period **2000-2020**.
- Choose the option for the entire available region.

**File location:** `/work/klok0126/landcover/`

**Example file name:** 
 - `ESACCI-LC-L4-LCCS-Map-300m-P1Y-2009-v2.0.7cds.nc`
 - `ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7cds.nc` 
 - `C3S-LC-L4-LCCS-Map-300m-P1Y-2016-v2.1.1.nc`
 - `C3S-LC-L4-LCCS-Map-300m-P1Y-2020-v2.1.1.nc` 

# EVI Dataset

Follow the instructions below to download the EVI (Enhanced Vegetation Index) dataset:

- Visit [NASA’s AppEEARS](https://appeears.earthdatacloud.nasa.gov/).
- Register and sign in if you haven’t already.
- Set the spatial filter with the following coordinates:
  - Latitude from **21.80°N** to **25.40°N**
  - Longitude from **119.90°E** to **122.10°E**
- Set the start date to **01-01-2000** and the end date to **12-31-2021**.
- Choose the product **MOD13A1** and select the layer **_500m_16_days_EVI**.
- Select **GeoTiff** as the file format for download.

**File location:** `/work/klok0126/other_env/EVI/`

**Example file names:**
- `MOD13A1.006__500m_16_days_EVI_doy2000049_aid0001.tif`
- `MOD13A1.006__500m_16_days_EVI_doy2021353_aid0001.tif`

# Elevation Dataset

To download the Elevation dataset, simply click the link below:

[WorldClim Elevation Data](https://geodata.ucdavis.edu/climate/worldclim/2_1/base/wc2.1_30s_elev.zip)

**File location:** `/work/klok0126/other_env/elev/`

**Example file name:** `wc2.1_30s_elev.tif`